# Machine Learning Engineer Nanodegree

## Capstone project: Exoplanet search (from Kaggle)

Student Name: Graciano Patino

Kaggle reference: https://www.kaggle.com/keplersmachines/kepler-labelled-time-series-data?/

The mission as stated in the Github (https://github.com/winterdelta/KeplerAI) is to build a classification algorithm for identifying if a particular time series input includes an exoplanet or not. It also mentions that a number of methods were tested: 1-D CNN in Torch7, XGBoost in R and PCA in Python. However, none of these methods provided strong results according to the kaggle and Github references. 

For this project, I would evaluate deep learning algorithms. Per paper in the paragraph (below), these algorithms appear to provide better results compared to the ones already tried as mentioned above.

    1)	Initially I would evaluate 1-D CNN using Keras instead of Torch7. 
    2)	Based on reference paper, I would try adding different number of layers and filters in combination with other CNN parameters. Details would be included in project report.
    3)	The output of the CNNs would be the input to one or more dense layers.
    4)	Performance of each model to be measured as per evaluation metrics section.
    5)	Per kaggle source the test set is confirmed to have 5 exoplanets. This will also be useful on checking performance of algorithms. If an algorithm is unable to identify exoplanets on then testing set, then model might not be good. 

Please that the list above of models considered is not meant to be exhaustive for all possible scenarios in deep learning algorithms. It might be the case that other deep learning algorithms might be considered later should the ones proposed (above) fail in identifying any exoplanet as expected.

Reference paper: IDENTIFYING EXOPLANETS WITH DEEP LEARNING: A FIVE PLANET RESONANT CHAIN AROUND KEPLER-80 AND AN EIGHTH PLANET AROUND KEPLER-90 
(https://www.cfa.harvard.edu/~avanderb/kepler90i.pdf)


#### Note

In this Jupyter Notebook: Applying Grid Search to identify parameters for network.


In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import itertools
from IPython.display import display # Allows the use of display() for DataFrames
from get_results import plot_roc_auc, confusion_matrix_com

# Pretty display for notebooks
%matplotlib inline

# Some Sklearn libraries are required
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils.fixes import signature
from sklearn.metrics import average_precision_score

In [2]:
# Load the training data from Exoplanet dataset
train_data = pd.read_csv('kepler/exoTrain.csv')

In [3]:
# Find dimensions of the train data
train_data.shape

(5087, 3198)

In [4]:
# Testing  if train_data has any null fields
testing = pd.isnull(train_data)
testing *= 1
testing2 = testing.sum()
testing2.sum() # If results is zero, then there a no fields with "null" value

0

In [5]:
# Getting X_train and y_train
# Using iloc to select data using position instead of label and converting to numpy array using values
X_train = train_data.iloc[:,1:].values 
y_train = train_data.iloc[:,0:1].values 

In [6]:
# Find dimensions of the X_train data
X_train.shape

(5087, 3197)

In [7]:
# Find dimensions of the labels (y_train) data
y_train.shape

(5087, 1)

In [8]:
# y_train: Label is 2 for exoplanet and 1 for non-exoplanet
y_train[:5]

array([[2],
       [2],
       [2],
       [2],
       [2]])

In [9]:
y_train -= 1 # Changing labels to: 1 for exoplanet and 0 for non-exoplanet

In [10]:
y_train[:5]

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [11]:
# Load the testing data from Exoplanet dataset
test_data = pd.read_csv('kepler/exoTest.csv')

In [12]:
# Find dimensions of the test data
test_data.shape

(570, 3198)

In [13]:
# Testing  if test_data has any "null" fields
testing = pd.isnull(test_data)
testing *= 1
testing2 = testing.sum()
testing2.sum() # If results is zero, then there a no fields with "null" value

0

In [14]:
# Getting X_test and y_test
# Using iloc to select data using position instead of label and converting to numpy array (using values)
X_test = test_data.iloc[:,1:].values
y_test = test_data.iloc[:,0:1].values

In [15]:
# Find dimensions of the X_test data
X_test.shape

(570, 3197)

In [16]:
# Find dimensions of the labels (y_test) data
y_test.shape

(570, 1)

In [17]:
# y_test: Label is 2 for exoplanet and 1 for non-exoplanet
y_test[:6]

array([[2],
       [2],
       [2],
       [2],
       [2],
       [1]])

In [18]:
y_test -= 1 # Changing labels to: 1 for exoplanet and 0 for non-exoplanet

In [19]:
y_test[:6]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [0]])

In [20]:
# Normalizing the data since it is not normalized according to Kaggle/Github
from sklearn.preprocessing import StandardScaler

In [21]:
# Checking X_train data
X_train

array([[  93.85,   83.81,   20.1 , ...,   61.42,    5.08,  -39.54],
       [ -38.88,  -33.83,  -58.54, ...,    6.46,   16.  ,   19.93],
       [ 532.64,  535.92,  513.73, ...,  -28.91,  -70.02,  -96.67],
       ..., 
       [ 273.39,  278.  ,  261.73, ...,   88.42,   79.07,   79.43],
       [   3.82,    2.09,   -3.29, ...,  -14.55,   -6.41,   -2.55],
       [ 323.28,  306.36,  293.16, ...,  -16.72,  -14.09,   27.82]])

In [22]:
# Tranposing X_train before applying scaling such that mean is zero and variance is one
X_train.transpose()

array([[  93.85,  -38.88,  532.64, ...,  273.39,    3.82,  323.28],
       [  83.81,  -33.83,  535.92, ...,  278.  ,    2.09,  306.36],
       [  20.1 ,  -58.54,  513.73, ...,  261.73,   -3.29,  293.16],
       ..., 
       [  61.42,    6.46,  -28.91, ...,   88.42,  -14.55,  -16.72],
       [   5.08,   16.  ,  -70.02, ...,   79.07,   -6.41,  -14.09],
       [ -39.54,   19.93,  -96.67, ...,   79.43,   -2.55,   27.82]])

In [23]:
# # Checking X_train data
X_test

array([[  1.19880000e+02,   1.00210000e+02,   8.64600000e+01, ...,
          3.57800000e+01,   2.69430000e+02,   5.77200000e+01],
       [  5.73659000e+03,   5.69998000e+03,   5.71716000e+03, ...,
         -2.36619000e+03,  -2.29486000e+03,  -2.03472000e+03],
       [  8.44480000e+02,   8.17490000e+02,   7.70070000e+02, ...,
         -1.62680000e+02,  -3.67900000e+01,   3.06300000e+01],
       ..., 
       [ -5.40100000e+01,  -4.41300000e+01,  -4.12300000e+01, ...,
          5.47000000e+00,   1.44600000e+01,   1.87000000e+01],
       [  9.13600000e+01,   8.56000000e+01,   4.88100000e+01, ...,
         -8.43000000e+00,  -6.48000000e+00,   1.76000000e+01],
       [  3.07119000e+03,   2.78253000e+03,   2.60869000e+03, ...,
         -2.77220000e+02,  -6.96300000e+01,   1.21560000e+02]])

In [24]:
# Tranposing X_test before applying scaling such that mean is zero and variance is one
X_test.transpose()

array([[  1.19880000e+02,   5.73659000e+03,   8.44480000e+02, ...,
         -5.40100000e+01,   9.13600000e+01,   3.07119000e+03],
       [  1.00210000e+02,   5.69998000e+03,   8.17490000e+02, ...,
         -4.41300000e+01,   8.56000000e+01,   2.78253000e+03],
       [  8.64600000e+01,   5.71716000e+03,   7.70070000e+02, ...,
         -4.12300000e+01,   4.88100000e+01,   2.60869000e+03],
       ..., 
       [  3.57800000e+01,  -2.36619000e+03,  -1.62680000e+02, ...,
          5.47000000e+00,  -8.43000000e+00,  -2.77220000e+02],
       [  2.69430000e+02,  -2.29486000e+03,  -3.67900000e+01, ...,
          1.44600000e+01,  -6.48000000e+00,  -6.96300000e+01],
       [  5.77200000e+01,  -2.03472000e+03,   3.06300000e+01, ...,
          1.87000000e+01,   1.76000000e+01,   1.21560000e+02]])

In [25]:
# Standardize features by removing the mean and scaling to unit variance (sklearn.preprocessing)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Output in a numpy.ndarray
X_test = scaler.fit_transform(X_test) # Output in a numpy.ndarray

In [26]:
X_train.transpose() # Transpose back to original dimensions

array([[-0.00235557, -0.00852774,  0.01804893, ...,  0.00599336,
        -0.00654212,  0.00831333],
       [-0.00205404, -0.0074516 ,  0.01868969, ...,  0.00685579,
        -0.00580352,  0.00815701],
       [-0.00579778, -0.00938685,  0.01673115, ...,  0.00523005,
        -0.00686528,  0.00666449],
       ..., 
       [ 0.0341983 ,  0.03109682,  0.02910084, ...,  0.03572195,
         0.0299112 ,  0.02978874],
       [ 0.02736753,  0.02803863,  0.02275218, ...,  0.03191466,
         0.0266614 ,  0.02618942],
       [ 0.01805157,  0.02216476,  0.01410023, ...,  0.02628002,
         0.02060995,  0.02271046]])

In [27]:
X_test.transpose() # Transpose back to original dimensions

array([[-0.03143654,  0.41497541,  0.02615413, ..., -0.04525719,
        -0.03370329,  0.20313139],
       [-0.05057432,  0.39343385,  0.00629912, ..., -0.0620191 ,
        -0.05173276,  0.16210798],
       [-0.03559448,  0.41363759,  0.01894572, ..., -0.04578193,
        -0.0385983 ,  0.16563567],
       ..., 
       [-0.00907555, -0.2624662 , -0.03001166, ..., -0.01227304,
        -0.01373939, -0.04209481],
       [ 0.00470934, -0.26591026, -0.02760726, ..., -0.02219864,
        -0.02440852, -0.03107299],
       [-0.00786554, -0.22375419, -0.01066057, ..., -0.01189145,
        -0.01200494, -0.00127881]])

In [28]:
# Fix random seed for reproducibility
seed = 10
np.random.seed(seed)

In [29]:
# Importing Keras libraries

from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPool1D, Dense, Dropout, Flatten
from keras.layers import BatchNormalization, Input, concatenate, Activation
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from keras.callbacks import ModelCheckpoint 

Using TensorFlow backend.


In [30]:
#Convert data into 3d tensor (Input 0 in Conv1D is incompatible with layer conv1d_1: expected ndim=3, found ndim=2)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [31]:
# Checking shape of X_test tensor
X_test.shape

(570, 3197, 1)

In [32]:
# Checking shape of X_train_new tensor
X_train.shape

(5087, 3197, 1)

#### Data exploration and preparation ended (above)

# Ending Data Preparation

# GRID SEARCH 

## Grid Search Implementation

Reference on using GridSearch for tuning Hyperparameters for DL models:

https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

## GRID SEARCH 8cnn, 2dnn

In [33]:
# Selecting a subset of the training set
X_GS1000 = X_train[0:1000,:]
y_GS1000 = y_train[0:1000,:]

In [34]:
# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create model, required for KerasClassifier
def create_model(optimizer='adam', dropout_rate=0.0):
    # create model
    model = Sequential()
    # Defining network architecture
    model.add(Conv1D(filters=12, kernel_size=6, activation='relu', input_shape=(3197,1)))
    model.add(Conv1D(filters=12, kernel_size=6, activation='relu'))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=24, kernel_size=6, activation='relu'))
    model.add(Conv1D(filters=24, kernel_size=6, activation='relu'))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=48, kernel_size=6, activation='relu'))
    model.add(Conv1D(filters=48, kernel_size=6, activation='relu'))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=96, kernel_size=6, activation='relu'))
    model.add(Conv1D(filters=96, kernel_size=6, activation='relu'))
    model.add(MaxPool1D(strides=4))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(96, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(optimizer=Adam(1e-5), loss="binary_crossentropy", metrics=["accuracy"])
    return model

# Fix random seed for reproducibility (done above on previous cells)
# Dataset was created in previous section


In [35]:
# Create model
model = KerasClassifier(build_fn=create_model, verbose=2)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 25, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_GS1000, y_GS1000)


Epoch 1/10
Epoch 1/10
Epoch 1/10
Epoch 1/25
15s - loss: 0.6824 - acc: 0.8979
Epoch 2/10
15s - loss: 0.6847 - acc: 0.8591
Epoch 2/10
15s - loss: 0.6876 - acc: 0.8561
Epoch 2/10
15s - loss: 0.6823 - acc: 0.9024
Epoch 2/25
15s - loss: 0.6462 - acc: 0.9940
Epoch 3/10
15s - loss: 0.6660 - acc: 0.9355
Epoch 3/10
15s - loss: 0.6693 - acc: 0.9325
Epoch 3/10
15s - loss: 0.6461 - acc: 0.9925
Epoch 3/25
15s - loss: 0.4838 - acc: 0.9970
Epoch 4/10
15s - loss: 0.5415 - acc: 0.9340
Epoch 4/10
15s - loss: 0.4834 - acc: 0.9970
Epoch 4/25
15s - loss: 0.5975 - acc: 0.9385
Epoch 4/10
15s - loss: 0.1372 - acc: 0.9925
Epoch 5/10
15s - loss: 0.2772 - acc: 0.9370
Epoch 5/10
15s - loss: 0.3850 - acc: 0.9415
Epoch 5/10
15s - loss: 0.1356 - acc: 0.9925
Epoch 5/25
15s - loss: 0.2433 - acc: 0.9385
Epoch 6/10
15s - loss: 0.0622 - acc: 0.9970
Epoch 6/10
15s - loss: 0.0607 - acc: 0.9970
Epoch 6/25
15s - loss: 0.2664 - acc: 0.9340
Epoch 6/10
14s - loss: 0.0481 - acc: 0.9970
Epoch 7/10
14s - loss: 0.2281 - acc: 0.9415

15s - loss: 0.0012 - acc: 1.0000
Epoch 38/50
15s - loss: 0.2042 - acc: 0.9415
Epoch 23/50
15s - loss: 0.0327 - acc: 1.0000
Epoch 12/100
15s - loss: 0.2246 - acc: 0.9430
Epoch 13/50
15s - loss: 0.0017 - acc: 1.0000
Epoch 39/50
15s - loss: 0.2164 - acc: 0.9430
Epoch 24/50
15s - loss: 0.0354 - acc: 0.9985
Epoch 13/100
15s - loss: 0.2228 - acc: 0.9400
Epoch 14/50
15s - loss: 0.0010 - acc: 1.0000
15s - loss: 0.2260 - acc: 0.9430
Epoch 25/50
Epoch 40/50
15s - loss: 0.0264 - acc: 1.0000
Epoch 14/100
15s - loss: 0.2065 - acc: 0.9460
Epoch 15/50
15s - loss: 0.0010 - acc: 1.0000
Epoch 41/50
15s - loss: 0.2278 - acc: 0.9430
Epoch 26/50
15s - loss: 0.0266 - acc: 1.0000
Epoch 15/100
15s - loss: 0.2184 - acc: 0.9430
Epoch 16/50
15s - loss: 0.0012 - acc: 1.0000
Epoch 42/50
15s - loss: 0.2049 - acc: 0.9430
Epoch 27/50
15s - loss: 0.0273 - acc: 0.9970
Epoch 16/100
15s - loss: 0.2239 - acc: 0.9430
Epoch 17/50
15s - loss: 0.2137 - acc: 0.9445
Epoch 28/50
15s - loss: 8.3802e-04 - acc: 1.0000
Epoch 43/50
1

Epoch 7/10
15s - loss: 3.4008e-04 - acc: 1.0000
Epoch 57/100
15s - loss: 0.2096 - acc: 0.9430
Epoch 18/100
14s - loss: 0.3335 - acc: 0.9910
Epoch 8/10
15s - loss: 0.2175 - acc: 0.9445
Epoch 33/100
15s - loss: 0.0011 - acc: 1.0000
Epoch 58/100
15s - loss: 0.2056 - acc: 0.9430
Epoch 19/100
14s - loss: 0.1978 - acc: 0.9925
Epoch 9/10
15s - loss: 0.2174 - acc: 0.9430
Epoch 34/100
15s - loss: 0.0013 - acc: 1.0000
Epoch 59/100
15s - loss: 0.2161 - acc: 0.9445
Epoch 20/100
14s - loss: 0.1480 - acc: 0.9940
Epoch 10/10
15s - loss: 0.2265 - acc: 0.9460
Epoch 35/100
15s - loss: 3.8898e-04 - acc: 1.0000
Epoch 60/100
15s - loss: 0.2135 - acc: 0.9430
Epoch 21/100
14s - loss: 0.1259 - acc: 0.9940
15s - loss: 0.2157 - acc: 0.9400
Epoch 36/100
15s - loss: 2.2924e-04 - acc: 1.0000
Epoch 61/100
14s - loss: 0.2135 - acc: 0.9445
Epoch 22/100
13s - loss: 0.2188 - acc: 0.9400
Epoch 37/100
Epoch 1/10
14s - loss: 4.4405e-04 - acc: 1.0000
Epoch 62/100
14s - loss: 0.2016 - acc: 0.9430
Epoch 23/100
14s - loss: 0.

15s - loss: 0.2071 - acc: 0.9445
Epoch 77/100
Epoch 1/25
13s - loss: 0.0425 - acc: 1.0000
Epoch 20/25
13s - loss: 0.1864 - acc: 0.9520
Epoch 63/100
14s - loss: 0.1879 - acc: 0.9475
Epoch 78/100
16s - loss: 0.6975 - acc: 0.3253
Epoch 2/25
14s - loss: 0.0402 - acc: 1.0000
Epoch 21/25
15s - loss: 0.1887 - acc: 0.9505
Epoch 64/100
15s - loss: 0.1887 - acc: 0.9490
Epoch 79/100
14s - loss: 0.6889 - acc: 0.7811
Epoch 3/25
14s - loss: 0.0458 - acc: 0.9970
Epoch 22/25
15s - loss: 0.1761 - acc: 0.9535
Epoch 65/100
15s - loss: 0.1986 - acc: 0.9445
Epoch 80/100
14s - loss: 0.0359 - acc: 0.9955
Epoch 23/25
14s - loss: 0.6763 - acc: 0.9220
Epoch 4/25
15s - loss: 0.1720 - acc: 0.9520
Epoch 66/100
15s - loss: 0.1991 - acc: 0.9505
Epoch 81/100
14s - loss: 0.0366 - acc: 0.9940
Epoch 24/25
14s - loss: 0.6554 - acc: 0.9295
Epoch 5/25
15s - loss: 0.1950 - acc: 0.9430
Epoch 67/100
15s - loss: 0.1876 - acc: 0.9475
Epoch 82/100
14s - loss: 0.0311 - acc: 0.9985
Epoch 25/25
14s - loss: 0.6201 - acc: 0.9265
Epoc

Epoch 23/50
14s - loss: 0.2366 - acc: 0.9400
Epoch 22/50
14s - loss: 0.2261 - acc: 0.9430
Epoch 15/50
14s - loss: 0.3587 - acc: 0.9835
Epoch 8/100
14s - loss: 0.0147 - acc: 0.9985
Epoch 24/50
14s - loss: 0.2240 - acc: 0.9415
Epoch 23/50
14s - loss: 0.2306 - acc: 0.9445
Epoch 16/50
14s - loss: 0.2554 - acc: 0.9925
Epoch 9/100
14s - loss: 0.0151 - acc: 0.9970
Epoch 25/50
14s - loss: 0.2395 - acc: 0.9400
Epoch 24/50
14s - loss: 0.2285 - acc: 0.9430
Epoch 17/50
14s - loss: 0.1588 - acc: 0.9895
Epoch 10/100
14s - loss: 0.0096 - acc: 1.0000
Epoch 26/50
14s - loss: 0.2405 - acc: 0.9400
Epoch 25/50
14s - loss: 0.2110 - acc: 0.9415
Epoch 18/50
14s - loss: 0.1099 - acc: 0.9940
Epoch 11/100
14s - loss: 0.0102 - acc: 1.0000
Epoch 27/50
14s - loss: 0.2261 - acc: 0.9415
Epoch 26/50
14s - loss: 0.2148 - acc: 0.9430
Epoch 19/50
14s - loss: 0.0617 - acc: 0.9970
Epoch 12/100
14s - loss: 0.0069 - acc: 1.0000
Epoch 28/50
14s - loss: 0.2176 - acc: 0.9430
Epoch 27/50
14s - loss: 0.2205 - acc: 0.9430
Epoch 2

...

Epoch 9/100
15s - loss: 0.6837 - acc: 0.9100
Epoch 5/100
14s - loss: 0.6861 - acc: 0.8574
Epoch 3/10
14s - loss: 0.6552 - acc: 0.9265
Epoch 10/100
15s - loss: 0.1274 - acc: 0.9985
Epoch 45/100
14s - loss: 0.6791 - acc: 0.9250
Epoch 6/100
14s - loss: 0.6827 - acc: 0.9520
Epoch 4/10
14s - loss: 0.6479 - acc: 0.9325
Epoch 11/100
15s - loss: 0.1063 - acc: 0.9970
Epoch 46/100
14s - loss: 0.6730 - acc: 0.9340
Epoch 7/100
14s - loss: 0.6805 - acc: 0.9715
Epoch 5/10
14s - loss: 0.6242 - acc: 0.9370
Epoch 12/100
14s - loss: 0.0940 - acc: 0.9910
Epoch 47/100
14s - loss: 0.6630 - acc: 0.9355
Epoch 8/100
14s - loss: 0.6773 - acc: 0.9850
Epoch 6/10
14s - loss: 0.6217 - acc: 0.9340
Epoch 13/100
14s - loss: 0.0605 - acc: 0.9970
Epoch 48/100
15s - loss: 0.6517 - acc: 0.9340
Epoch 9/100
14s - loss: 0.6682 - acc: 0.9895
Epoch 7/10
14s - loss: 0.5976 - acc: 0.9340
Epoch 14/100
15s - loss: 0.0630 - acc: 0.9970
Epoch 49/100
14s - loss: 0.6350 - acc: 0.9310
Epoch 10/100
14s - loss: 0.6642 - acc: 0.9940
Epoc

Epoch 51/100
15s - loss: 0.2305 - acc: 0.9415
Epoch 56/100
14s - loss: 0.0061 - acc: 1.0000
Epoch 91/100
14s - loss: 0.6215 - acc: 0.9880
Epoch 14/25
14s - loss: 0.2212 - acc: 0.9445
Epoch 52/100
14s - loss: 0.2124 - acc: 0.9415
Epoch 57/100
14s - loss: 0.0056 - acc: 1.0000
Epoch 92/100
14s - loss: 0.5994 - acc: 0.9865
Epoch 15/25
14s - loss: 0.2113 - acc: 0.9460
Epoch 53/100
14s - loss: 0.2138 - acc: 0.9415
Epoch 58/100
14s - loss: 0.0074 - acc: 0.9985
Epoch 93/100
14s - loss: 0.5829 - acc: 0.9970
Epoch 16/25
14s - loss: 0.2128 - acc: 0.9400
Epoch 54/100
15s - loss: 0.2258 - acc: 0.9400
Epoch 59/100
15s - loss: 0.0067 - acc: 0.9985
Epoch 94/100
14s - loss: 0.5671 - acc: 0.9940
Epoch 17/25
14s - loss: 0.2173 - acc: 0.9430
Epoch 55/100
14s - loss: 0.2104 - acc: 0.9400
Epoch 60/100
14s - loss: 0.0052 - acc: 1.0000
Epoch 95/100
14s - loss: 0.5739 - acc: 0.9820
Epoch 18/25
15s - loss: 0.2117 - acc: 0.9430
Epoch 56/100
14s - loss: 0.2089 - acc: 0.9445
Epoch 61/100
14s - loss: 0.0033 - acc: 

Epoch 7/50
13s - loss: 0.2075 - acc: 0.9490
Epoch 99/100
12s - loss: 0.6326 - acc: 0.9955
Epoch 10/50
13s - loss: 0.6719 - acc: 0.9430
Epoch 8/50
23s - loss: 0.6956 - acc: 0.3208
Epoch 2/50
15s - loss: 0.2075 - acc: 0.9460
Epoch 100/100
14s - loss: 0.6237 - acc: 0.9880
Epoch 11/50
14s - loss: 0.6611 - acc: 0.9400
Epoch 9/50
15s - loss: 0.6936 - acc: 0.5472
Epoch 3/50
14s - loss: 0.2135 - acc: 0.9445
14s - loss: 0.6176 - acc: 0.9940
Epoch 12/50
14s - loss: 0.6559 - acc: 0.9430
Epoch 10/50
13s - loss: 0.6920 - acc: 0.7916
Epoch 4/50
13s - loss: 0.5922 - acc: 0.9925
Epoch 13/50
13s - loss: 0.6504 - acc: 0.9400
Epoch 11/50
Epoch 1/100
13s - loss: 0.6906 - acc: 0.9040
Epoch 5/50
12s - loss: 0.5780 - acc: 0.9970
Epoch 14/50
12s - loss: 0.6481 - acc: 0.9430
Epoch 12/50
13s - loss: 0.6895 - acc: 0.9130
Epoch 6/50
14s - loss: 0.5544 - acc: 0.9910
Epoch 15/50
21s - loss: 0.6869 - acc: 0.9324
Epoch 2/100
14s - loss: 0.6308 - acc: 0.9430
Epoch 13/50
14s - loss: 0.6878 - acc: 0.9250
Epoch 7/50
14s 

Epoch 48/50
14s - loss: 0.1003 - acc: 0.9955
Epoch 44/100
14s - loss: 0.6910 - acc: 0.8141
Epoch 5/100
14s - loss: 0.6903 - acc: 0.7796
Epoch 3/100
14s - loss: 0.2203 - acc: 0.9445
Epoch 49/50
14s - loss: 0.0727 - acc: 1.0000
Epoch 45/100
14s - loss: 0.6903 - acc: 0.8576
Epoch 6/100
15s - loss: 0.6880 - acc: 0.9010
Epoch 4/100
14s - loss: 0.2235 - acc: 0.9430
Epoch 50/50
14s - loss: 0.0605 - acc: 1.0000
Epoch 46/100
14s - loss: 0.6872 - acc: 0.9160
Epoch 7/100
15s - loss: 0.6851 - acc: 0.9055
Epoch 5/100
14s - loss: 0.2200 - acc: 0.9385
13s - loss: 0.6847 - acc: 0.9235
Epoch 8/100
14s - loss: 0.0559 - acc: 0.9985
Epoch 47/100
13s - loss: 0.6841 - acc: 0.9190
Epoch 6/100
13s - loss: 0.6843 - acc: 0.9235
Epoch 9/100
13s - loss: 0.0399 - acc: 1.0000
Epoch 48/100
12s - loss: 0.6795 - acc: 0.9130
Epoch 7/100
11s - loss: 0.6804 - acc: 0.9220
Epoch 10/100
11s - loss: 0.0413 - acc: 0.9985
Epoch 49/100
11s - loss: 0.6750 - acc: 0.9190
Epoch 8/100
11s - loss: 0.6779 - acc: 0.9295
Epoch 11/100
10

7s - loss: 0.2447 - acc: 0.9370
Epoch 64/100
7s - loss: 0.2115 - acc: 0.9445
Epoch 62/100
7s - loss: 0.2246 - acc: 0.9445
Epoch 65/100
7s - loss: 0.2149 - acc: 0.9445
Epoch 63/100
7s - loss: 0.2429 - acc: 0.9400
Epoch 66/100
7s - loss: 0.2162 - acc: 0.9445
Epoch 64/100
7s - loss: 0.2265 - acc: 0.9430
Epoch 67/100
7s - loss: 0.2134 - acc: 0.9430
Epoch 65/100
7s - loss: 0.2225 - acc: 0.9400
Epoch 68/100
7s - loss: 0.2298 - acc: 0.9430
Epoch 66/100
7s - loss: 0.2210 - acc: 0.9445
Epoch 69/100
7s - loss: 0.2095 - acc: 0.9430
Epoch 67/100
7s - loss: 0.2283 - acc: 0.9400
Epoch 70/100
7s - loss: 0.2277 - acc: 0.9415
Epoch 68/100
7s - loss: 0.2308 - acc: 0.9385
Epoch 71/100
7s - loss: 0.2254 - acc: 0.9445
Epoch 69/100
7s - loss: 0.2115 - acc: 0.9475
Epoch 72/100
7s - loss: 0.2221 - acc: 0.9430
Epoch 70/100
7s - loss: 0.2300 - acc: 0.9355
Epoch 73/100
7s - loss: 0.2184 - acc: 0.9460
Epoch 71/100
7s - loss: 0.2362 - acc: 0.9430
Epoch 74/100
7s - loss: 0.2313 - acc: 0.9445
Epoch 72/100
7s - loss:

In [36]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.963000 using {'epochs': 10, 'batch_size': 80}
0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 10}
0.962000 (0.051554) with: {'epochs': 25, 'batch_size': 10}
0.961000 (0.050831) with: {'epochs': 50, 'batch_size': 10}
0.957000 (0.048558) with: {'epochs': 100, 'batch_size': 10}
0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 20}
0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 20}
0.961000 (0.050890) with: {'epochs': 50, 'batch_size': 20}
0.956000 (0.049955) with: {'epochs': 100, 'batch_size': 20}
0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 40}
0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 40}
0.958000 (0.049091) with: {'epochs': 50, 'batch_size': 40}
0.956000 (0.049471) with: {'epochs': 100, 'batch_size': 40}
0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 60}
0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 60}
0.963000 (0.052248) with: {'epochs': 50, 'batch_size': 60}
0.962000 (0.051554) with: {'epochs': 100, 'batch_size': 60

### Run1: Best: 0.963000 using {'batch_size': 80, 'epochs': 50} (3cnn 2dnn)

    0.962000 (0.051554) with: {'batch_size': 10, 'epochs': 10}
    0.963000 (0.052248) with: {'batch_size': 10, 'epochs': 25}
    0.962000 (0.051554) with: {'batch_size': 10, 'epochs': 50}
    0.963000 (0.052248) with: {'batch_size': 10, 'epochs': 100}
    0.963000 (0.052248) with: {'batch_size': 20, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 20, 'epochs': 25}
    0.959000 (0.049657) with: {'batch_size': 20, 'epochs': 50}
    0.663000 (0.204708) with: {'batch_size': 20, 'epochs': 100}
    0.963000 (0.052248) with: {'batch_size': 40, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 40, 'epochs': 25}
    0.959000 (0.049657) with: {'batch_size': 40, 'epochs': 50}
    0.716000 (0.326578) with: {'batch_size': 40, 'epochs': 100}
    0.962000 (0.051554) with: {'batch_size': 60, 'epochs': 10}
    0.961000 (0.050890) with: {'batch_size': 60, 'epochs': 25}
    0.962000 (0.051554) with: {'batch_size': 60, 'epochs': 50}
    0.960000 (0.050258) with: {'batch_size': 60, 'epochs': 100}
    0.962000 (0.051554) with: {'batch_size': 80, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 80, 'epochs': 25}
    0.963000 (0.052248) with: {'batch_size': 80, 'epochs': 50}
    0.961000 (0.050890) with: {'batch_size': 80, 'epochs': 100}
    0.962000 (0.051554) with: {'batch_size': 100, 'epochs': 10}
    0.962000 (0.051554) with: {'batch_size': 100, 'epochs': 25}
    0.962000 (0.051554) with: {'batch_size': 100, 'epochs': 50}
    0.961000 (0.050890) with: {'batch_size': 100, 'epochs': 100}


### Run 2: Best: 0.963000 using {'epochs': 10, 'batch_size': 80} (3cnn 2dnn)
    0.960000 (0.050258) with: {'epochs': 10, 'batch_size': 10}
    0.962000 (0.051554) with: {'epochs': 25, 'batch_size': 10}
    0.961000 (0.050890) with: {'epochs': 50, 'batch_size': 10}
    0.960000 (0.050258) with: {'epochs': 100, 'batch_size': 10}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 20}
    0.960000 (0.050138) with: {'epochs': 25, 'batch_size': 20}
    0.958000 (0.049091) with: {'epochs': 50, 'batch_size': 20}
    0.662000 (0.402405) with: {'epochs': 100, 'batch_size': 20}
    0.962000 (0.051554) with: {'epochs': 10, 'batch_size': 40}
    0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 40}
    0.954000 (0.047181) with: {'epochs': 50, 'batch_size': 40}
    0.960000 (0.050258) with: {'epochs': 100, 'batch_size': 40}
    0.961000 (0.050890) with: {'epochs': 10, 'batch_size': 60}
    0.961000 (0.050890) with: {'epochs': 25, 'batch_size': 60}
    0.963000 (0.052248) with: {'epochs': 50, 'batch_size': 60}
    0.962000 (0.051554) with: {'epochs': 100, 'batch_size': 60}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 80}
    0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 80}
    0.962000 (0.051554) with: {'epochs': 50, 'batch_size': 80}
    0.962000 (0.051554) with: {'epochs': 100, 'batch_size': 80}
    0.962000 (0.051554) with: {'epochs': 10, 'batch_size': 100}
    0.962000 (0.051554) with: {'epochs': 25, 'batch_size': 100}
    0.962000 (0.051554) with: {'epochs': 50, 'batch_size': 100}
    0.962000 (0.051554) with: {'epochs': 100, 'batch_size': 100}


### Run 3: Best: 0.963000 using {'epochs': 10, 'batch_size': 80} (8 cnn 2 dnn)
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 10}
    0.962000 (0.051554) with: {'epochs': 25, 'batch_size': 10}
    0.961000 (0.050831) with: {'epochs': 50, 'batch_size': 10}
    0.957000 (0.048558) with: {'epochs': 100, 'batch_size': 10}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 20}
    0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 20}
    0.961000 (0.050890) with: {'epochs': 50, 'batch_size': 20}
    0.956000 (0.049955) with: {'epochs': 100, 'batch_size': 20}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 40}
    0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 40}
    0.958000 (0.049091) with: {'epochs': 50, 'batch_size': 40}
    0.956000 (0.049471) with: {'epochs': 100, 'batch_size': 40}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 60}
    0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 60}
    0.963000 (0.052248) with: {'epochs': 50, 'batch_size': 60}
    0.962000 (0.051554) with: {'epochs': 100, 'batch_size': 60}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 80}
    0.963000 (0.052248) with: {'epochs': 25, 'batch_size': 80}
    0.962000 (0.051554) with: {'epochs': 50, 'batch_size': 80}
    0.958000 (0.049091) with: {'epochs': 100, 'batch_size': 80}
    0.963000 (0.052248) with: {'epochs': 10, 'batch_size': 100}
    0.962000 (0.051554) with: {'epochs': 25, 'batch_size': 100}
    0.963000 (0.052248) with: {'epochs': 50, 'batch_size': 100}
    0.963000 (0.052248) with: {'epochs': 100, 'batch_size': 100}